In [ ]:
!pip install faiss-cpu
!pip install urllib
!pip install huggingface_hub
!pip install langchain
!pip install unstructured
!pip install --upgrade --quiet  text-generation
!pip install langchainhub
!pip install sentencepiece
!pip install langchain-community
!pip install tiktoken
!pip install sentence-transformers

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as Soup
import requests
import re
import json
from langchain_community.document_loaders import JSONLoader
from pathlib import Path
from pprint import pprint
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceHub
import os
import getpass
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS
import numpy as np

In [ ]:
inference_api_key = getpass.getpass("Enter your HF Inference API Key:\n\n")

Enter your HF Inference API Key:

··········


In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

Обозначаем словарь с Ключом - Название серии, Значением - ссылкой, по которой будем парсить описание

In [ ]:
series = {"Биби и его папа": "https://losyash-library.fandom.com/ru/wiki/%D0%91%D0%B8%D0%B1%D0%B8_%D0%B8_%D0%B5%D0%B3%D0%BE_%D0%BF%D0%B0%D0%BF%D0%B0",
          "Маскарад": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D0%B0%D1%81%D0%BA%D0%B0%D1%80%D0%B0%D0%B4",
          "Каникулы Биби": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D0%B0%D0%BD%D0%B8%D0%BA%D1%83%D0%BB%D1%8B_%D0%91%D0%B8%D0%B1%D0%B8",
           "Личная жизнь": "https://losyash-library.fandom.com/ru/wiki/%D0%9B%D0%B8%D1%87%D0%BD%D0%B0%D1%8F_%D0%B6%D0%B8%D0%B7%D0%BD%D1%8C",
           "Её звали Нюша": "https://losyash-library.fandom.com/ru/wiki/%D0%95%D1%91_%D0%B7%D0%B2%D0%B0%D0%BB%D0%B8_%D0%9D%D1%8E%D1%88%D0%B0",
          "Взаперти": "https://losyash-library.fandom.com/ru/wiki/%D0%92%D0%B7%D0%B0%D0%BF%D0%B5%D1%80%D1%82%D0%B8",
          "Край Земли": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D1%80%D0%B0%D0%B9_%D0%97%D0%B5%D0%BC%D0%BB%D0%B8",
          "Полоса невезения": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%BE%D0%BB%D0%BE%D1%81%D0%B0_%D0%BD%D0%B5%D0%B2%D0%B5%D0%B7%D0%B5%D0%BD%D0%B8%D1%8F",
           "Кулинария": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D1%83%D0%BB%D0%B8%D0%BD%D0%B0%D1%80%D0%B8%D1%8F",
          "Комната грусти": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D0%BE%D0%BC%D0%BD%D0%B0%D1%82%D0%B0_%D0%B3%D1%80%D1%83%D1%81%D1%82%D0%B8",
          "Вестибулярный аппарат": "https://losyash-library.fandom.com/ru/wiki/%D0%92%D0%B5%D1%81%D1%82%D0%B8%D0%B1%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BF%D0%BF%D0%B0%D1%80%D0%B0%D1%82",
          "Смысл жизни": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%BC%D1%8B%D1%81%D0%BB_%D0%B6%D0%B8%D0%B7%D0%BD%D0%B8",
          "Лабиринт": "https://losyash-library.fandom.com/ru/wiki/%D0%9B%D0%B0%D0%B1%D0%B8%D1%80%D0%B8%D0%BD%D1%82",
          #"Комната смеха": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D0%BE%D0%BC%D0%BD%D0%B0%D1%82%D0%B0_%D1%81%D0%BC%D0%B5%D1%85%D0%B0",
          "Самое главное": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%B0%D0%BC%D0%BE%D0%B5_%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%BE%D0%B5",
          "Утерянные извинения": "https://losyash-library.fandom.com/ru/wiki/%D0%A3%D1%82%D0%B5%D1%80%D1%8F%D0%BD%D0%BD%D1%8B%D0%B5_%D0%B8%D0%B7%D0%B2%D0%B8%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F",
          "Право на одиночество": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D1%80%D0%B0%D0%B2%D0%BE_%D0%BD%D0%B0_%D0%BE%D0%B4%D0%B8%D0%BD%D0%BE%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%BE",
          "Не может быть": "https://losyash-library.fandom.com/ru/wiki/%D0%9D%D0%B5_%D0%BC%D0%BE%D0%B6%D0%B5%D1%82_%D0%B1%D1%8B%D1%82%D1%8C",
          "1 апреля": "https://losyash-library.fandom.com/ru/wiki/1_%D0%B0%D0%BF%D1%80%D0%B5%D0%BB%D1%8F",
          #"Без никого": "https://losyash-library.fandom.com/ru/wiki/%D0%91%D0%B5%D0%B7_%D0%BD%D0%B8%D0%BA%D0%BE%D0%B3%D0%BE",
          #"Комментатор": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D0%BE%D0%BC%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%82%D0%BE%D1%80",
          "Вишнёвый сад": "https://losyash-library.fandom.com/ru/wiki/%D0%92%D0%B8%D1%88%D0%BD%D1%91%D0%B2%D1%8B%D0%B9_%D1%81%D0%B0%D0%B4",
          "Красная книга": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%B0%D1%8F_%D0%BA%D0%BD%D0%B8%D0%B3%D0%B0",
          "Индийский чай": "https://losyash-library.fandom.com/ru/wiki/%D0%98%D0%BD%D0%B4%D0%B8%D0%B9%D1%81%D0%BA%D0%B8%D0%B9_%D1%87%D0%B0%D0%B9",
          "Ода для комода": "https://losyash-library.fandom.com/ru/wiki/%D0%9E%D0%B4%D0%B0_%D0%B4%D0%BB%D1%8F_%D0%BA%D0%BE%D0%BC%D0%BE%D0%B4%D0%B0",
          "Картография": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D0%B0%D1%80%D1%82%D0%BE%D0%B3%D1%80%D0%B0%D1%84%D0%B8%D1%8F",
          "Лёд": "https://losyash-library.fandom.com/ru/wiki/%D0%9B%D1%91%D0%B4",
          "Хоккей": "https://losyash-library.fandom.com/ru/wiki/%D0%A5%D0%BE%D0%BA%D0%BA%D0%B5%D0%B9",
         "Музотерапия": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D1%83%D0%B7%D0%BE%D1%82%D0%B5%D1%80%D0%B0%D0%BF%D0%B8%D1%8F",
          #"О, благодарная!": "https://losyash-library.fandom.com/ru/wiki/%D0%9E,_%D0%B1%D0%BB%D0%B0%D0%B3%D0%BE%D0%B4%D0%B0%D1%80%D0%BD%D0%B0%D1%8F",
          "Эрудит": "https://losyash-library.fandom.com/ru/wiki/%D0%AD%D1%80%D1%83%D0%B4%D0%B8%D1%82",
           "Щедрое небо": "https://losyash-library.fandom.com/ru/wiki/%D0%A9%D0%B5%D0%B4%D1%80%D0%BE%D0%B5_%D0%BD%D0%B5%D0%B1%D0%BE",
          "Солнечный зайчик": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%BE%D0%BB%D0%BD%D0%B5%D1%87%D0%BD%D1%8B%D0%B9_%D0%B7%D0%B0%D0%B9%D1%87%D0%B8%D0%BA",
          "Ты - есть": "https://losyash-library.fandom.com/ru/wiki/%D0%A2%D1%8B_%E2%80%94_%D0%B5%D1%81%D1%82%D1%8C",
          "Медиум": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D0%B5%D0%B4%D0%B8%D1%83%D0%BC",
          "Психолог": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D1%81%D0%B8%D1%85%D0%BE%D0%BB%D0%BE%D0%B3",
          "Инкогнито": "https://losyash-library.fandom.com/ru/wiki/%D0%98%D0%BD%D0%BA%D0%BE%D0%B3%D0%BD%D0%B8%D1%82%D0%BE",
          "На память": "https://losyash-library.fandom.com/ru/wiki/%D0%9D%D0%B0_%D0%BF%D0%B0%D0%BC%D1%8F%D1%82%D1%8C",
           "Чистый спорт": "https://losyash-library.fandom.com/ru/wiki/%D0%A7%D0%B8%D1%81%D1%82%D1%8B%D0%B9_%D1%81%D0%BF%D0%BE%D1%80%D1%82",
          "Парашют": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%B0%D1%80%D0%B0%D1%88%D1%8E%D1%82",
          "Шахматы": "https://losyash-library.fandom.com/ru/wiki/%D0%A8%D0%B0%D1%85%D0%BC%D0%B0%D1%82%D1%8B",
          "Сыр-бор Новый-год": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D1%8B%D1%80-%D0%B1%D0%BE%D1%80_%D0%9D%D0%BE%D0%B2%D1%8B%D0%B9-%D0%B3%D0%BE%D0%B4",
          "Сплетники": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%BF%D0%BB%D0%B5%D1%82%D0%BD%D0%B8%D0%BA%D0%B8",
          "Вальхалла": "https://losyash-library.fandom.com/ru/wiki/%D0%92%D0%B0%D0%BB%D1%8C%D1%85%D0%B0%D0%BB%D0%BB%D0%B0",
          "Фан-клуб Козетты": "https://losyash-library.fandom.com/ru/wiki/%D0%A4%D0%B0%D0%BD-%D0%BA%D0%BB%D1%83%D0%B1_%D0%9A%D0%BE%D0%B7%D0%B5%D1%82%D1%82%D1%8B",
           "Волшебный кувшинчик": "https://losyash-library.fandom.com/ru/wiki/%D0%92%D0%BE%D0%BB%D1%88%D0%B5%D0%B1%D0%BD%D1%8B%D0%B9_%D0%BA%D1%83%D0%B2%D1%88%D0%B8%D0%BD%D1%87%D0%B8%D0%BA",
          "Сила бездействия": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%B8%D0%BB%D0%B0_%D0%B1%D0%B5%D0%B7%D0%B4%D0%B5%D0%B9%D1%81%D1%82%D0%B2%D0%B8%D1%8F",
          "Золотой век": "https://losyash-library.fandom.com/ru/wiki/%D0%97%D0%BE%D0%BB%D0%BE%D1%82%D0%BE%D0%B9_%D0%B2%D0%B5%D0%BA",
          "Старинный новогодний обычай": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D1%82%D0%B0%D1%80%D0%B8%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BD%D0%BE%D0%B2%D0%BE%D0%B3%D0%BE%D0%B4%D0%BD%D0%B8%D0%B9_%D0%BE%D0%B1%D1%8B%D1%87%D0%B0%D0%B9",
          "Нервный и заразный": "https://losyash-library.fandom.com/ru/wiki/%D0%9D%D0%B5%D1%80%D0%B2%D0%BD%D1%8B%D0%B9_%D0%B8_%D0%B7%D0%B0%D1%80%D0%B0%D0%B7%D0%BD%D1%8B%D0%B9",
          "Сказки": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%BA%D0%B0%D0%B7%D0%BA%D0%B8",
          "Лекарство от всего": "https://losyash-library.fandom.com/ru/wiki/%D0%9B%D0%B5%D0%BA%D0%B0%D1%80%D1%81%D1%82%D0%B2%D0%BE_%D0%BE%D1%82_%D0%B2%D1%81%D0%B5%D0%B3%D0%BE",
          "Здравствуй, мой дневничок!": "https://losyash-library.fandom.com/ru/wiki/%D0%97%D0%B4%D1%80%D0%B0%D0%B2%D1%81%D1%82%D0%B2%D1%83%D0%B9,_%D0%BC%D0%BE%D0%B9_%D0%B4%D0%BD%D0%B5%D0%B2%D0%BD%D0%B8%D1%87%D0%BE%D0%BA!",
          "Подходящий момент": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%BE%D0%B4%D1%85%D0%BE%D0%B4%D1%8F%D1%89%D0%B8%D0%B9_%D0%BC%D0%BE%D0%BC%D0%B5%D0%BD%D1%82",
          "Меткая стрела": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D0%B5%D1%82%D0%BA%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B5%D0%BB%D0%B0",
           "Оракул": "https://losyash-library.fandom.com/ru/wiki/%D0%9E%D1%80%D0%B0%D0%BA%D1%83%D0%BB",
          "Справочное бюро": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%BF%D1%80%D0%B0%D0%B2%D0%BE%D1%87%D0%BD%D0%BE%D0%B5_%D0%B1%D1%8E%D1%80%D0%BE",
          "Традиция": "https://losyash-library.fandom.com/ru/wiki/%D0%A2%D1%80%D0%B0%D0%B4%D0%B8%D1%86%D0%B8%D1%8F",
          "Паранормальное приключение": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%B0%D1%80%D0%B0%D0%BD%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%BF%D1%80%D0%B8%D0%BA%D0%BB%D1%8E%D1%87%D0%B5%D0%BD%D0%B8%D0%B5",
          "Просто так и вопреки": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D1%80%D0%BE%D1%81%D1%82%D0%BE_%D1%82%D0%B0%D0%BA_%D0%B8_%D0%B2%D0%BE%D0%BF%D1%80%D0%B5%D0%BA%D0%B8",
          "Жажда": "https://losyash-library.fandom.com/ru/wiki/%D0%96%D0%B0%D0%B6%D0%B4%D0%B0",
          "Гусений": "https://losyash-library.fandom.com/ru/wiki/%D0%93%D1%83%D1%81%D0%B5%D0%BD%D0%B8%D0%B9",
          "Переводчик": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%B5%D1%80%D0%B5%D0%B2%D0%BE%D0%B4%D1%87%D0%B8%D0%BA",
          "Церемониймейстер": "https://losyash-library.fandom.com/ru/wiki/%D0%A6%D0%B5%D1%80%D0%B5%D0%BC%D0%BE%D0%BD%D0%B8%D0%B9%D0%BC%D0%B5%D0%B9%D1%81%D1%82%D0%B5%D1%80",
           "Большая книга снов": "https://losyash-library.fandom.com/ru/wiki/%D0%91%D0%BE%D0%BB%D1%8C%D1%88%D0%B0%D1%8F_%D0%BA%D0%BD%D0%B8%D0%B3%D0%B0_%D1%81%D0%BD%D0%BE%D0%B2",
          "День учителя": "https://losyash-library.fandom.com/ru/wiki/%D0%94%D0%B5%D0%BD%D1%8C_%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D1%8F",
          "Очки": "https://losyash-library.fandom.com/ru/wiki/%D0%9E%D1%87%D0%BA%D0%B8",
          "Морская капуста": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D0%BE%D1%80%D1%81%D0%BA%D0%B0%D1%8F_%D0%BA%D0%B0%D0%BF%D1%83%D1%81%D1%82%D0%B0",
          "Причастность": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D1%80%D0%B8%D1%87%D0%B0%D1%81%D1%82%D0%BD%D0%BE%D1%81%D1%82%D1%8C",
          "Зимняя сказка": "https://losyash-library.fandom.com/ru/wiki/%D0%97%D0%B8%D0%BC%D0%BD%D1%8F%D1%8F_%D1%81%D0%BA%D0%B0%D0%B7%D0%BA%D0%B0",
          "Охота на тигра": "https://losyash-library.fandom.com/ru/wiki/%D0%9E%D1%85%D0%BE%D1%82%D0%B0_%D0%BD%D0%B0_%D1%82%D0%B8%D0%B3%D1%80%D0%B0",
          "Правила тенниса": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D1%80%D0%B0%D0%B2%D0%B8%D0%BB%D0%B0_%D1%82%D0%B5%D0%BD%D0%BD%D0%B8%D1%81%D0%B0",
          "Зажигательное шоу": "https://losyash-library.fandom.com/ru/wiki/%D0%97%D0%B0%D0%B6%D0%B8%D0%B3%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D1%88%D0%BE%D1%83",
          "Адаптиция": "https://losyash-library.fandom.com/ru/wiki/%D0%90%D0%B4%D0%B0%D0%BF%D1%82%D0%B8%D1%86%D0%B8%D1%8F",
           "Окрошка": "https://losyash-library.fandom.com/ru/wiki/%D0%9E%D0%BA%D1%80%D0%BE%D1%88%D0%BA%D0%B0",
          "Зверинец": "https://losyash-library.fandom.com/ru/wiki/%D0%97%D0%B2%D0%B5%D1%80%D0%B8%D0%BD%D0%B5%D1%86",
          "Синяя птица": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%B8%D0%BD%D1%8F%D1%8F_%D0%BF%D1%82%D0%B8%D1%86%D0%B0",
          "Сияние марципанов": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%B8%D1%8F%D0%BD%D0%B8%D0%B5_%D0%BC%D0%B0%D1%80%D1%86%D0%B8%D0%BF%D0%B0%D0%BD%D0%BE%D0%B2",
          "Славная повесть о лучезарном рыцаре": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%BB%D0%B0%D0%B2%D0%BD%D0%B0%D1%8F_%D0%BF%D0%BE%D0%B2%D0%B5%D1%81%D1%82%D1%8C_%D0%BE_%D0%BB%D1%83%D1%87%D0%B5%D0%B7%D0%B0%D1%80%D0%BD%D0%BE%D0%BC_%D1%80%D1%8B%D1%86%D0%B0%D1%80%D0%B5",
          "Игра по-взрослому": "https://losyash-library.fandom.com/ru/wiki/%D0%98%D0%B3%D1%80%D0%B0_%D0%BF%D0%BE-%D0%B2%D0%B7%D1%80%D0%BE%D1%81%D0%BB%D0%BE%D0%BC%D1%83",
          "Сторож": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D1%82%D0%BE%D1%80%D0%BE%D0%B6",
          "Супержульбист": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D1%83%D0%BF%D0%B5%D1%80%D0%B6%D1%83%D0%BB%D1%8C%D0%B1%D0%B8%D1%81%D1%82",
          "Обратная связь": "https://losyash-library.fandom.com/ru/wiki/%D0%9E%D0%B1%D1%80%D0%B0%D1%82%D0%BD%D0%B0%D1%8F_%D1%81%D0%B2%D1%8F%D0%B7%D1%8C",
          "Самолёт и бабочки": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%B0%D0%BC%D0%BE%D0%BB%D1%91%D1%82_%D0%B8_%D0%B1%D0%B0%D0%B1%D0%BE%D1%87%D0%BA%D0%B8",
          "Лягушатник": "https://losyash-library.fandom.com/ru/wiki/%D0%9B%D1%8F%D0%B3%D1%83%D1%88%D0%B0%D1%82%D0%BD%D0%B8%D0%BA",
           "Погребок": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%BE%D0%B3%D1%80%D0%B5%D0%B1%D0%BE%D0%BA",
          "Тапёр": "https://losyash-library.fandom.com/ru/wiki/%D0%A2%D0%B0%D0%BF%D1%91%D1%80",
          "Ляпус фикус хряпус": "https://losyash-library.fandom.com/ru/wiki/%D0%9B%D1%8F%D0%BF%D1%83%D1%81_%D1%84%D0%B8%D0%BA%D1%83%D1%81_%D1%85%D1%80%D1%8F%D0%BF%D1%83%D1%81",
          "Копатыч и волна": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D0%BE%D0%BF%D0%B0%D1%82%D1%8B%D1%87_%D0%B8_%D0%B2%D0%BE%D0%BB%D0%BD%D0%B0",
          "Аквапарк": "https://losyash-library.fandom.com/ru/wiki/%D0%90%D0%BA%D0%B2%D0%B0%D0%BF%D0%B0%D1%80%D0%BA",
          "Шерсть": "https://losyash-library.fandom.com/ru/wiki/%D0%A8%D0%B5%D1%80%D1%81%D1%82%D1%8C",
          "Внутреннее пространство": "https://losyash-library.fandom.com/ru/wiki/%D0%92%D0%BD%D1%83%D1%82%D1%80%D0%B5%D0%BD%D0%BD%D0%B5%D0%B5_%D0%BF%D1%80%D0%BE%D1%81%D1%82%D1%80%D0%B0%D0%BD%D1%81%D1%82%D0%B2%D0%BE",
          "Шляпа": "https://losyash-library.fandom.com/ru/wiki/%D0%A8%D0%BB%D1%8F%D0%BF%D0%B0",
          "Кораблик": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D0%BE%D1%80%D0%B0%D0%B1%D0%BB%D0%B8%D0%BA",
          "Тихие лесные тропинки": "https://losyash-library.fandom.com/ru/wiki/%D0%A2%D0%B8%D1%85%D0%B8%D0%B5_%D0%BB%D0%B5%D1%81%D0%BD%D1%8B%D0%B5_%D1%82%D1%80%D0%BE%D0%BF%D0%B8%D0%BD%D0%BA%D0%B8",
          "Принцесса цирка": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D1%80%D0%B8%D0%BD%D1%86%D0%B5%D1%81%D1%81%D0%B0_%D1%86%D0%B8%D1%80%D0%BA%D0%B0",
           "Остров": "https://losyash-library.fandom.com/ru/wiki/%D0%9E%D1%81%D1%82%D1%80%D0%BE%D0%B2",
          "Культура": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D1%83%D0%BB%D1%8C%D1%82%D1%83%D1%80%D0%B0",
          "Музей": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D1%83%D0%B7%D0%B5%D0%B9",
          "Прибытие поезда": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D1%80%D0%B8%D0%B1%D1%8B%D1%82%D0%B8%D0%B5_%D0%BF%D0%BE%D0%B5%D0%B7%D0%B4%D0%B0",
          "Старики и море": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D1%82%D0%B0%D1%80%D0%B8%D0%BA%D0%B8_%D0%B8_%D0%BC%D0%BE%D1%80%D0%B5",
          "Лунатики": "https://losyash-library.fandom.com/ru/wiki/%D0%9B%D1%83%D0%BD%D0%B0%D1%82%D0%B8%D0%BA%D0%B8",
          "Подлёдный лов": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%BE%D0%B4%D0%BB%D1%91%D0%B4%D0%BD%D1%8B%D0%B9_%D0%BB%D0%BE%D0%B2",
          "Девушка в песках": "https://losyash-library.fandom.com/ru/wiki/%D0%94%D0%B5%D0%B2%D1%83%D1%88%D0%BA%D0%B0_%D0%B2_%D0%BF%D0%B5%D1%81%D0%BA%D0%B0%D1%85",
          "Наваждение": "https://losyash-library.fandom.com/ru/wiki/%D0%9D%D0%B0%D0%B2%D0%B0%D0%B6%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5",
          "Повод для жизни": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%BE%D0%B2%D0%BE%D0%B4_%D0%B4%D0%BB%D1%8F_%D0%B6%D0%B8%D0%B7%D0%BD%D0%B8",
          "Неизвестный кот в ящике": "https://losyash-library.fandom.com/ru/wiki/%D0%9D%D0%B5%D0%B8%D0%B7%D0%B2%D0%B5%D1%81%D1%82%D0%BD%D1%8B%D0%B9_%D0%BA%D0%BE%D1%82_%D0%B2_%D1%8F%D1%89%D0%B8%D0%BA%D0%B5",
          "Труба": "https://losyash-library.fandom.com/ru/wiki/%D0%A2%D1%80%D1%83%D0%B1%D0%B0",
          "Берег там!": "https://losyash-library.fandom.com/ru/wiki/%D0%91%D0%B5%D1%80%D0%B5%D0%B3_%D1%82%D0%B0%D0%BC!",
          "Душеприказчики": "https://losyash-library.fandom.com/ru/wiki/%D0%94%D1%83%D1%88%D0%B5%D0%BF%D1%80%D0%B8%D0%BA%D0%B0%D0%B7%D1%87%D0%B8%D0%BA%D0%B8",
          "Музыка": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D1%83%D0%B7%D1%8B%D0%BA%D0%B0",
          "Скамейка": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%BA%D0%B0%D0%BC%D0%B5%D0%B9%D0%BA%D0%B0",
          "Принц для Нюши": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D1%80%D0%B8%D0%BD%D1%86_%D0%B4%D0%BB%D1%8F_%D0%9D%D1%8E%D1%88%D0%B8",
          "Некультурный": "https://losyash-library.fandom.com/ru/wiki/%D0%9D%D0%B5%D0%BA%D1%83%D0%BB%D1%8C%D1%82%D1%83%D1%80%D0%BD%D1%8B%D0%B9",
          "Как собрать друзей по-быстрому": "https://losyash-library.fandom.com/ru/wiki/%D0%9A%D0%B0%D0%BA_%D1%81%D0%BE%D0%B1%D1%80%D0%B0%D1%82%D1%8C_%D0%B4%D1%80%D1%83%D0%B7%D0%B5%D0%B9_%D0%BF%D0%BE-%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%BE%D0%BC%D1%83",
          "Новые зубы Кроша": "https://losyash-library.fandom.com/ru/wiki/%D0%9D%D0%BE%D0%B2%D1%8B%D0%B5_%D0%B7%D1%83%D0%B1%D1%8B_%D0%9A%D1%80%D0%BE%D1%88%D0%B0",
          "Магнетизм": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D0%B0%D0%B3%D0%BD%D0%B5%D1%82%D0%B8%D0%B7%D0%BC",
          "Горы и конфеты": "https://losyash-library.fandom.com/ru/wiki/%D0%93%D0%BE%D1%80%D1%8B_%D0%B8_%D0%BA%D0%BE%D0%BD%D1%84%D0%B5%D1%82%D1%8B",
          "Живые часы": "https://losyash-library.fandom.com/ru/wiki/%D0%96%D0%B8%D0%B2%D1%8B%D0%B5_%D1%87%D0%B0%D1%81%D1%8B",
          "Это сладкое слово мёд": "https://losyash-library.fandom.com/ru/wiki/%D0%AD%D1%82%D0%BE_%D1%81%D0%BB%D0%B0%D0%B4%D0%BA%D0%BE%D0%B5_%D1%81%D0%BB%D0%BE%D0%B2%D0%BE_%C2%AB%D0%BC%D1%91%D0%B4%C2%BB",
          "Страшилка для Нюши": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D1%82%D1%80%D0%B0%D1%88%D0%B8%D0%BB%D0%BA%D0%B0_%D0%B4%D0%BB%D1%8F_%D0%9D%D1%8E%D1%88%D0%B8",
          "Маленькое большое море": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D0%B0%D0%BB%D0%B5%D0%BD%D1%8C%D0%BA%D0%BE%D0%B5_%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%B5_%D0%BC%D0%BE%D1%80%D0%B5",
          "Археология": "https://losyash-library.fandom.com/ru/wiki/%D0%90%D1%80%D1%85%D0%B5%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%8F",
          "Танцор диско": "https://losyash-library.fandom.com/ru/wiki/%D0%A2%D0%B0%D0%BD%D1%86%D0%BE%D1%80_%D0%B4%D0%B8%D1%81%D0%BA%D0%BE",
          "Мыльная опера": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D1%8B%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D0%BE%D0%BF%D0%B5%D1%80%D0%B0",
          "Футбол": "https://losyash-library.fandom.com/ru/wiki/%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB",
          "ОРЗ": "https://losyash-library.fandom.com/ru/wiki/%D0%9E%D0%A0%D0%97",
          "Снотворец": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%BD%D0%BE%D1%82%D0%B2%D0%BE%D1%80%D0%B5%D1%86",
          "День справедливости": "https://losyash-library.fandom.com/ru/wiki/%D0%94%D0%B5%D0%BD%D1%8C_%D1%81%D0%BF%D1%80%D0%B0%D0%B2%D0%B5%D0%B4%D0%BB%D0%B8%D0%B2%D0%BE%D1%81%D1%82%D0%B8",
          "Большой куш": "https://losyash-library.fandom.com/ru/wiki/%D0%91%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%B9_%D0%BA%D1%83%D1%88",
          #"Власть моды": "https://losyash-library.fandom.com/ru/wiki/%D0%92%D0%BB%D0%B0%D1%81%D1%82%D1%8C_%D0%BC%D0%BE%D0%B4%D1%8B",
          "Печенье судьбы": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%B5%D1%87%D0%B5%D0%BD%D1%8C%D0%B5_%D1%81%D1%83%D0%B4%D1%8C%D0%B1%D1%8B",
          "Детский сад": "https://losyash-library.fandom.com/ru/wiki/%D0%94%D0%B5%D1%82%D1%81%D0%BA%D0%B8%D0%B9_%D1%81%D0%B0%D0%B4",
          "Наследство": "https://losyash-library.fandom.com/ru/wiki/%D0%9D%D0%B0%D1%81%D0%BB%D0%B5%D0%B4%D1%81%D1%82%D0%B2%D0%BE"}

In [ ]:
series1 = {"Переводчик": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%B5%D1%80%D0%B5%D0%B2%D0%BE%D0%B4%D1%87%D0%B8%D0%BA",
          "Маскарад": "https://losyash-library.fandom.com/ru/wiki/%D0%9C%D0%B0%D1%81%D0%BA%D0%B0%D1%80%D0%B0%D0%B4",
          "Полоса невезения": "https://losyash-library.fandom.com/ru/wiki/%D0%9F%D0%BE%D0%BB%D0%BE%D1%81%D0%B0_%D0%BD%D0%B5%D0%B2%D0%B5%D0%B7%D0%B5%D0%BD%D0%B8%D1%8F",
           "Личная жизнь": "https://losyash-library.fandom.com/ru/wiki/%D0%9B%D0%B8%D1%87%D0%BD%D0%B0%D1%8F_%D0%B6%D0%B8%D0%B7%D0%BD%D1%8C",
           "Смысл жизни": "https://losyash-library.fandom.com/ru/wiki/%D0%A1%D0%BC%D1%8B%D1%81%D0%BB_%D0%B6%D0%B8%D0%B7%D0%BD%D0%B8"}

In [ ]:
def parser(url):
    res = requests.get(url)
    soup = Soup(res.text, 'html.parser')
    start = soup.find(id='Подробный_сюжет')
    stop = soup.find(id='Персонажи')
    result = ""
    for element in start.next_elements:
        if element.name == 'p':
            result += element.text
            #print(element.text)
        if element == stop:
            break
    result = re.sub(r'[\n\xa0]', ' ', result)
    return result

new_data = {key: parser(value).strip() for key, value in series.items()}
clean_data = {key: re.sub(r'[\n\xa0]', ' ', value).strip() for key, value in new_data.items()}

In [ ]:
import pandas as pd
test = {'question': ['Сколько Нюш было на маскараде?', 'Кто из cмешариков оделся кроликом?', 'Почему Совунья бежала по Ромашковой долине?', 'Что Копатыч пытался скрыть от Совуньи на своём огороде?', 'Кто такой Карыч?', 'Что такое Кузинатра?', 'Почему Копатыч решил проверить, правда ли Карычу не везет?', 'Что Копатыч сделал для Карыча?', 'Почему Лосяш решил записывать слова Кроша и Ёжика?', 'Что Лосяш сделал с помощью словаря тютюрского языка?'],
        'ground_truth': ['На маскараде было шесть Нюш.', 'Копатыч оделся кроликом.', 'Совунья бежала по Ромашковой долине, потому что она с воодушевлением напевала бодрую мелодию и наслаждалась летним днем.', 'Копатыч пытался скрыть от Совуньи странный металлический столб, установленный посреди двора.', 'Карыч - это ворон, который пытается помочь Барашу найти смысл жизни.', 'Кузинатра - это то, что дает смысл жизни.', 'Копатыч решил проверить, правда ли Карычу не везет, потому что он подметил, что если ему не везет, то остальным вокруг него должно везти, и он может этим приносить окружающим пользу.', 'Копатыч сделал для Карыча защитный костюм, который должен был защитить его от пчёл на пасеке.', 'Лосяш решил записывать слова Кроша и Ёжика, потому что они говорили на неизвестном ему языке, и он хотел понять, о чём они говорят', 'Лосяш смог поговорить с Крошем и Ёжиком на их языке, используя словарь.'],
        'contexts': ['Но когда они зашли, то удивились сами — перед ними сидело шесть Нюш! Одна из них говорит голосом Карыча: «О, ещё два оригинала… Где маникюр делали, девочки?». Тут в дом входит Бараш, и, видя много-много Нюш, падает в обморок.', 'Копатыч, увидев его, обиделся — его лось заставил одеться скромным кроликом, а сам учёный пойдёт в ярком и прекрасном костюме — костюме Нюши!', 'В летний день по Ромашковой долине бежит Совунья, с воодушевлением напевающая бодрую мелодию. Она проносится мимо сакуры и ручья, затем преодолевает луг и, проломив забор, врывается в огород Копатыча, где спотыкается и падает на траву.', 'Сам медведь в это время занят каким-то делом: не переставая нервно озираться по сторонам, он пытается прикрыть листвой странный металлический столб, установленный посреди двора.', 'На совет не лежать на снегу, чтобы не простудиться, Бараш отвечает: «А смысл не простужаться?». Далее диалог продолжается, Карыч понимает, что Барашу необходим смысл. Он решает поведать Барашу о Кузинатре, дающей смысл.', 'Карыч понимает, что Барашу необходим смысл. Он решает поведать Барашу о Кузинатре, дающей смысл.', 'Копатыч, выплюнув сливу, говорит ворону, что если ему не везёт, то остальным вокруг него должно везти, ведь он же, по мнению Копатыча, притягивает к себе всю неудачу и может этим приносить окружающим пользу.', 'Медведь, выставляя свои шахматные фигуры на исходные позиции, говорит другу, чтобы он не говорил ерунды, но ворон даёт ему такой ответ: «Это не ерунда. Это... ПОЛОСА НЕВЕЗЕНИЯ! Чтобы я не делал, всё идёт наперекосяк. Никогда такого не видел.»', 'Лосяш останавливается, прислушивается к разговору друзей и… не понимает ни слова! Крош и Ёжик говорят на каком-то странном, неизвестном лосю языке.', 'Лосяш подходит к друзьям и, пользуясь словарём, начинает говорить с ними на тютюрском языке.'],
}

b = pd.DataFrame(test)
b

,question,ground_truth,contexts
0,Сколько Нюш было на маскараде?,На маскараде было шесть Нюш.,"Но когда они зашли, то удивились сами — перед ..."
1,Кто из cмешариков оделся кроликом?,Копатыч оделся кроликом.,"Копатыч, увидев его, обиделся — его лось заста..."
2,Почему Совунья бежала по Ромашковой долине?,"Совунья бежала по Ромашковой долине, потому чт...",В летний день по Ромашковой долине бежит Совун...
3,Что Копатыч пытался скрыть от Совуньи на своём...,Копатыч пытался скрыть от Совуньи странный мет...,Сам медведь в это время занят каким-то делом: ...
4,Кто такой Карыч?,"Карыч - это ворон, который пытается помочь Бар...","На совет не лежать на снегу, чтобы не простуди..."
5,Что такое Кузинатра?,"Кузинатра - это то, что дает смысл жизни.","Карыч понимает, что Барашу необходим смысл. Он..."
6,"Почему Копатыч решил проверить, правда ли Кары...","Копатыч решил проверить, правда ли Карычу не в...","Копатыч, выплюнув сливу, говорит ворону, что е..."
7,Что Копатыч сделал для Карыча?,"Копатыч сделал для Карыча защитный костюм, кот...","Медведь, выставляя свои шахматные фигуры на ис..."
8,Почему Лосяш решил записывать слова Кроша и Ёж...,"Лосяш решил записывать слова Кроша и Ёжика, по...","Лосяш останавливается, прислушивается к разгов..."
9,Что Лосяш сделал с помощью словаря тютюрского ...,Лосяш смог поговорить с Крошем и Ёжиком на их ...,"Лосяш подходит к друзьям и, пользуясь словарём..."


In [ ]:
with open('data.json', 'w', encoding='utf-8') as json_file:
    json.dump(clean_data, json_file, ensure_ascii=False, indent=4)

In [ ]:
loader = TextLoader("data.json")
documents = loader.load()
# documents1 = loader.load()
# documents1

[Document(metadata={'source': 'data.json'}, page_content='{\n    "Переводчик": "В один солнечный летний день Лосяш отправляется с сачком ловить бабочек. Во время погони за одной из них он замечает Кроша и Ёжика, идущих по тропинке и о чём-то разговаривающих. Лось останавливается, прислушивается к разговору друзей и… не понимает ни слова! Крош и Ёжик говорят на каком-то странном, неизвестном лосю языке. Лосяш в недоумении провожает взглядом друзей, и тут в его голову приходит интересная мысль… В это время Крош с Ёжиком мастерят на пляже водный велосипед из подручных средств, продолжая разговаривать на том же непонятном языке. Но тут к ним подходит Лосяш и, прося тех разговаривать помедленнее, что-то записывает в блокнот. Друзья заглядывают в записи лося и видят там слова из своего языка, после чего спрашивают у Лосяша, зачем же ему это понадобилось. Лось отвечает, что ему это нужно для науки. «А, ну давай, записывай» — говорит Ёжик. И тогда Лосяш начинает следовать за кроликом и ежом: о

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=1400,
    chunk_overlap=100,
)
chunks = text_splitter.split_documents(documents)

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name='intfloat/multilingual-e5-large')
db = FAISS.from_documents(chunks, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
PROMPT_TEMPLATE = """
Ты дружелюбный чат-бот, который отвечает на вопросы про вселенную Смешариков с опорой на предоставленный контекст:

{context}

---

Ответь на вопрос с учётом предоставленного контекста: {question}
"""

query_text = "Что Лосяш сделал с помощью словаря тютюрского языка в серии Переводчик?"
results = db.similarity_search_with_relevance_scores(query_text, k=3, score_threshold=0.65)
# if len(results) == 0 or results[0][1] < 0.7:
#    print(f"Не могу найти результат 🤖.")


context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
#print(prompt)

llm = HuggingFaceHub(
      huggingfacehub_api_token = inference_api_key,
      repo_id= "HuggingFaceH4/zephyr-7b-beta",
      task="text-generation",
      model_kwargs={
          "max_new_tokens": 512,
          "top_k": 30,
          "temperature": 0.1,
          "repetition_penalty": 1.03,
      },
  )

response_text = llm.predict(prompt)

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}"
print(formatted_response)

Response: Human: 
Ты дружелюбный чат-бот, который отвечает на вопросы про вселенную Смешариков с опорой на предоставленный контекст:

"Переводчик": "В один солнечный летний день Лосяш отправляется с сачком ловить бабочек. Во время погони за одной из них он замечает Кроша и Ёжика, идущих по тропинке и о чём-то разговаривающих. Лось останавливается, прислушивается к разговору друзей и… не понимает ни слова! Крош и Ёжик говорят на каком-то странном, неизвестном лосю языке. Лосяш в недоумении провожает взглядом друзей, и тут в его голову приходит интересная мысль… В это время Крош с Ёжиком мастерят на пляже водный велосипед из подручных средств, продолжая разговаривать на том же непонятном языке. Но тут к ним подходит Лосяш и, прося тех разговаривать помедленнее, что-то записывает в блокнот. Друзья заглядывают в записи лося и видят там слова из своего языка, после чего спрашивают у Лосяша, зачем же ему это понадобилось. Лось отвечает, что ему это нужно для науки. «А, ну давай, записывай» —

#Semantic Textual Similarity, BERTScore и BM25+

In [ ]:
chatbot_responses = ['Шесть Нюш было на маскараде.', 'Копатыч оделся кроликом, но он был не очень красивым и довольно нелепым', 'Совунья бежала по Ромашковой долине в поисках ромашек, чтобы сделать из них венок, после того как ей ежедневно дарили по одной ромашке в тридцать дней.', 'Копатыч пытался скрыть от Совуньи странный металлический столб, который он присвоил для какой-то неизвестной цели, возможно, связанной с нашествием инопланетян.',
                            'Карыч - это друг Лосяши, часто участвующий в диалогах и спорах с другими персонажами, такими как Бараш и Крош. В сцене "Смысл жизни" он помогает Барашу найти смысл в жизни, рассказывая ему о Кузинатре.', 'Кузинатра - мистическое место, которое упоминается в сновидении Бараша во время его путешествия с Кар-Карычем. Бараш считает, что Кузинатра даёт смысл жизни, и он просит Кар-Карыча отвести его к ней. Однако, в конце концов, становится ясно, что они вернулись на то же место, откуда вышли, и Бараш понимает, что смысл жизни всё-таки есть.',
                           'Копатыч решил проверить, правда ли Карычу не везет в серии полоса невезения, потому что он заметил, что у Карыча началась череда неудачных событий, начиная с того, что он на бегу споткнулся об корень и упал в лужу, затем он упал в реку над обрывом, пытаясь перейти её по бревну, и дальше усталый ворон, ничего не видя, ударился головой об ветку дерева.',
                           'Копатыч попытался утешить Карыча, говоря, не все ж ему выигравать, но Карыч расстроенно ответил: «Да нет, ну с самого утра!». После этих слов нам показывают неудачный день Карыча: сначала ворон спросонья упал с постели, затем в ванной поток воды сбил его с ног, потом он разбил кружку, порвал газету и провалился в кресло. А после обеда Карыч пошел за грибами, но ему попалась целая поляна мухоморов. Потом, как назло для Карыча, начался дождь, и ворон спрятался под деревом, но в него ударила молния. И дальше у Карыча началась череда невезений: сначала он на бегу споткнулся об корень и упал в лужу, затем он упал в реку над обрывом, пытаясь перейти её по бревну. Ну а в конце концов, усталый ворон, ничего не видя, ударился головой об ветку дерева. После рассказа Карыча, Копатыч ставит последнюю шахматную фигуру, говоря ворону: «Шах! И мат...». После слов медведя, музыкант осматривает шахматную доску, видя, что она вся заставлена чёрными фигурами Копатыча, и что в нижнем левом углу стоит одинокий белый король. Осмотрев происходящее, ворон делает вывод: «Похоже, и жизнь мне поставила мат.». Медведь, выставляя свои шахматные фигуры на исходные позиции, говорит другу, чтобы он не говорил ерун',
                           'Лосяш решил записывать слова Кроша и Ёжика в серии Переводчик для науки, поско никто не знал языка, на котором они разговаривали. Он считал, что этот язык может помочь сохранить связь поколений.',
                           'Лосяш составил словарь тютюрского языка для сохранения связи поколений']

ground_truth = ['На маскараде было шесть Нюш.', 'Копатыч оделся кроликом.', 'Совунья бежала по Ромашковой долине, потому что она с воодушевлением напевала бодрую мелодию и наслаждалась летним днем.', 'Копатыч пытался скрыть от Совуньи странный металлический столб, установленный посреди двора.', 'Карыч - это ворон, который пытается помочь Барашу найти смысл жизни.', 'Кузинатра - это то, что дает смысл жизни.', 'Копатыч решил проверить, правда ли Карычу не везет, потому что он подметил, что если ему не везет, то остальным вокруг него должно везти, и он может этим приносить окружающим пользу.', 'Копатыч сделал для Карыча защитный костюм, который должен был защитить его от пчёл на пасеке.', 'Лосяш решил записывать слова Кроша и Ёжика, потому что они говорили на неизвестном ему языке, и он хотел понять, о чём они говорят', 'Лосяш смог поговорить с Крошем и Ёжиком на их языке, используя словарь.']
question = ['Сколько Нюш было на маскараде?', 'Кто из cмешариков оделся кроликом?', 'Почему Совунья бежала по Ромашковой долине?', 'Что Копатыч пытался скрыть от Совуньи на своём огороде?', 'Кто такой Карыч?', 'Что такое Кузинатра?', 'Почему Копатыч решил проверить, правда ли Карычу не везет?', 'Что Копатыч сделал для Карыча?', 'Почему Лосяш решил записывать слова Кроша и Ёжика?', 'Что Лосяш сделал с помощью словаря тютюрского языка?']

Semantic Textual Similarity

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings1 = model.encode(ground_truth)
embeddings2 = model.encode(chatbot_responses)

similarities = model.similarity(embeddings1, embeddings2)
STS = []

for idx in range(len(ground_truth)):
    score = similarities[idx][idx]
    STS.append(score)
    print(f"Pair {idx + 1}:")
    print(f" - Sentence 1: '{ground_truth[idx]}'")
    print(f" - Sentence 2: '{chatbot_responses[idx]}'")
    print(f" - Similarity Score: {score:.4f}\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Pair 1:
 - Sentence 1: 'На маскараде было шесть Нюш.'
 - Sentence 2: 'Шесть Нюш было на маскараде.'
 - Similarity Score: 0.9726

Pair 2:
 - Sentence 1: 'Копатыч оделся кроликом.'
 - Sentence 2: 'Копатыч оделся кроликом, но он был не очень красивым и довольно нелепым'
 - Similarity Score: 0.8618

Pair 3:
 - Sentence 1: 'Совунья бежала по Ромашковой долине, потому что она с воодушевлением напевала бодрую мелодию и наслаждалась летним днем.'
 - Sentence 2: 'Совунья бежала по Ромашковой долине в поисках ромашек, чтобы сделать из них венок, после того как ей ежедневно дарили по одной ромашке в тридцать дней.'
 - Similarity Score: 0.8662

Pair 4:
 - Sentence 1: 'Копатыч пытался скрыть от Совуньи странный металлический столб, установленный посреди двора.'
 - Sentence 2: 'Копатыч пытался скрыть от Совуньи странный металлический столб, который он присвоил для какой-то неизвестной цели, возможно, связанной с нашествием инопланетян.'
 - Similarity Score: 0.8410

Pair 5:
 - Sentence 1: 'Карыч - эт

In [ ]:
a = [round(t.item(), 4) for t in STS]
a

[0.9726, 0.8618, 0.8662, 0.841, 0.7264, 0.5055, 0.57, 0.4249, 0.836, 0.7843]

BERTScore

In [ ]:
!pip install bert-score

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer

# BERTScore calculation
scorer = BERTScorer(model_type='bert-base-uncased')
BERTScore = []

for idx in range(len(ground_truth)):
    ref = [ground_truth[idx]]
    cand = [chatbot_responses[idx]]
    P, R, F1 = scorer.score(cand, ref)
    BERTScore.append(round(F1.item(), 4))
    print(f"Pair {idx + 1}:")
    print(f" - Ground_truth: '{ground_truth[idx]}'")
    print(f" - Chatbot_response: '{chatbot_responses[idx]}'")
    print(f" - BERTScore Precision: {P.item():.4f}, Recall: {R.item():.4f}, F1: {F1.item():.4f}\n")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Pair 1:
 - Ground_truth: 'На маскараде было шесть Нюш.'
 - Chatbot_response: 'Шесть Нюш было на маскараде.'
 - BERTScore Precision: 0.9190, Recall: 0.9190, F1: 0.9190

Pair 2:
 - Ground_truth: 'Копатыч оделся кроликом.'
 - Chatbot_response: 'Копатыч оделся кроликом, но он был не очень красивым и довольно нелепым'
 - BERTScore Precision: 0.7324, Recall: 0.9335, F1: 0.8208

Pair 3:
 - Ground_truth: 'Совунья бежала по Ромашковой долине, потому что она с воодушевлением напевала бодрую мелодию и наслаждалась летним днем.'
 - Chatbot_response: 'Совунья бежала по Ромашковой долине в поисках ромашек, чтобы сделать из них венок, после того как ей ежедневно дарили по одной ромашке в тридцать дней.'
 - BERTScore Precision: 0.7801, Recall: 0.7919, F1: 0.7859

Pair 4:
 - Ground_truth: 'Копатыч пытался скрыть от Совуньи странный металлический столб, установленный посреди двора.'
 - Chatbot_response: 'Копатыч пытался скрыть от Совуньи странный металлический столб, который он присвоил для какой-то неи

In [ ]:
BERTScore

[0.919, 0.8208, 0.7859, 0.8341, 0.7713, 0.7227, 0.7952, 0.7205, 0.7995, 0.6952]

BM25+

In [ ]:
!pip install bm25s
!pip install PyStemmer

In [ ]:
import bm25s
import Stemmer

corpus = []

# optional: create a stemmer
stemmer = Stemmer.Stemmer("russian")

for i in range(len(ground_truth)):
    corpus.append(ground_truth[i])
    corpus.append(chatbot_responses[i])

    query = question[i]
    query_tokens = bm25s.tokenize(query, stemmer=stemmer)

    corpus_tokens = bm25s.tokenize(corpus, stemmer=stemmer)

    retriever = bm25s.BM25(method="bm25+", delta=1.5)
    retriever.index(corpus_tokens)

    results, scores = retriever.retrieve(query_tokens, corpus=corpus, k=2)

    print(f"Вопрос: {query}")
    for j in range(results.shape[1]):
        doc, score = results[0, j], scores[0, j]
        print(f"Ранг {j+1} (оценка: {score:.2f}): {doc}")

    corpus.clear()

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/2 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/2 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


...

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/2 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/2 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Вопрос: Что Лосяш сделал с помощью словаря тютюрского языка?
Ранг 1 (оценка: 5.84): Лосяш составил словарь тютюрского языка для сохранения связи поколений
Ранг 2 (оценка: 4.66): Лосяш смог поговорить с Крошем и Ёжиком на их языке, используя словарь.


In [ ]:
results = pd.DataFrame({'question': question, 'ground_truth': ground_truth, 'chatbot_responses': chatbot_responses,
           'STS': a, 'BERTScore F1': BERTScore})
results

,question,ground_truth,chatbot_responses,STS,BERTScore F1
0,Сколько Нюш было на маскараде?,На маскараде было шесть Нюш.,Шесть Нюш было на маскараде.,0.9726,0.9190
1,Кто из cмешариков оделся кроликом?,Копатыч оделся кроликом.,"Копатыч оделся кроликом, но он был не очень кр...",0.8618,0.8208
2,Почему Совунья бежала по Ромашковой долине?,"Совунья бежала по Ромашковой долине, потому чт...",Совунья бежала по Ромашковой долине в поисках ...,0.8662,0.7859
3,Что Копатыч пытался скрыть от Совуньи на своём...,Копатыч пытался скрыть от Совуньи странный мет...,Копатыч пытался скрыть от Совуньи странный мет...,0.8410,0.8341
4,Кто такой Карыч?,"Карыч - это ворон, который пытается помочь Бар...","Карыч - это друг Лосяши, часто участвующий в д...",0.7264,0.7713
5,Что такое Кузинатра?,"Кузинатра - это то, что дает смысл жизни.","Кузинатра - мистическое место, которое упомина...",0.5055,0.7227
6,"Почему Копатыч решил проверить, правда ли Кары...","Копатыч решил проверить, правда ли Карычу не в...","Копатыч решил проверить, правда ли Карычу не в...",0.5700,0.7952
7,Что Копатыч сделал для Карыча?,"Копатыч сделал для Карыча защитный костюм, кот...","Копатыч попытался утешить Карыча, говоря, не в...",0.4249,0.7205
8,Почему Лосяш решил записывать слова Кроша и Ёж...,"Лосяш решил записывать слова Кроша и Ёжика, по...",Лосяш решил записывать слова Кроша и Ёжика в с...,0.8360,0.7995
9,Что Лосяш сделал с помощью словаря тютюрского ...,Лосяш смог поговорить с Крошем и Ёжиком на их ...,Лосяш составил словарь тютюрского языка для со...,0.7843,0.6952


#RAGAS

In [ ]:
# используем вектора OpenAI
embeddings_model = HuggingFaceInferenceAPIEmbeddings(api_key = inference_api_key, model_name= 'intfloat/multilingual-e5-large')

# разбиваем документ на чанки
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1400)
chunks = text_splitter.split_documents(smesh_docs)

# индексируем в векторном хранилище
vectorstore = FAISS.from_documents(chunks, embeddings_model)

llm = HuggingFaceHub(
      huggingfacehub_api_token = inference_api_key,
      repo_id= "HuggingFaceH4/zephyr-7b-beta",
      task="text-generation",
      model_kwargs={
          "max_new_tokens": 512,
          "top_k": 10,
          "temperature": 0.1,
          "repetition_penalty": 1.03,
      },
  )

# задаем параметры ретривера
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True
)

result = qa_chain({'query': 'Кто папа у Биби?'})

In [ ]:
import pandas as pd
from datasets import Dataset
from tqdm import tqdm

from ragas.metrics import (
    context_recall,
    context_precision,
    answer_relevancy,
    faithfulness,
)
from ragas import evaluate
from langchain.chains import RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings


text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=1400,
    chunk_overlap=100,
)
chunks = text_splitter.split_documents(documents)

embeddings_model = SentenceTransformerEmbeddings(model_name='intfloat/multilingual-e5-large')
vectorstore = FAISS.from_documents(chunks, embeddings_model)


llm = HuggingFaceHub(
      huggingfacehub_api_token = inference_api_key,
      repo_id= "HuggingFaceH4/zephyr-7b-beta",
      task="text-generation",
      model_kwargs={
          "max_new_tokens": 512,
          "top_k": 30,
          "temperature": 0.1,
          "repetition_penalty": 1.03,
      },
  )

# задаем параметры ретривера
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True
)

# Ответ нашей LLM (gpt-4) возьмем за ground truth
#validation_set = test_set[['question', 'answer']].rename(columns={'answer' : 'ground_truth'})

# создаем датасет для подачи в ragas
def create_data_for_ragas(qa_chain, eval_dataset):
    rag_dataset = []
    for index, row in tqdm(eval_dataset.iterrows(), total=eval_dataset.shape[0]):
        answer = qa_chain({'query': row['question']})
        rag_dataset.append(
            {"question" : row['question'],
            "answer" : answer['result'],
            "contexts" : [context.page_content for context in answer['source_documents']],
            "ground_truth" : row['ground_truth'][0] if isinstance(row['ground_truth'], list) else row['ground_truth']
            })
    rag_df = pd.DataFrame(rag_dataset)
    rag_eval_dataset = Dataset.from_pandas(rag_df)
    return rag_df

qa_ragas_baseline = create_data_for_ragas(qa_chain, test_set)




100%|██████████| 10/10 [00:00<00:00, 16.62it/s]
